### Data Cleaning Reliability Project

In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime

pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

In [2]:
# Load the dataset 
pd.set_option('display.max_columns', None)
df = pd.read_csv("/Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/complete_feat_dataset.csv") 


df["step_count2"] = df["act_walking_ep_0"] + df["act_running_ep_0"] # Create step_count2

In [43]:
df # 35507 rows
len(np.unique(df["uid"])) # This also includes test IDs

313

In [44]:
# Select Relevant Columns  (List of variables from Table 2 (based on preregistration)) 
variables_table2 = [ 
    # Sleep 
    'gm_sleep_duration',"sleep_duration", 'gm_sleep_duration_awake', 
    'gm_sleep_duration_deep', 'gm_sleep_duration_rem', 'gm_sleep_quality', 
 
    # Activity 
    'gm_dailies_step', 'step_count2', 'garmin_steps', 
    'act_still_ep_0', 'gm_dailies_active_kcal', 'gm_dailies_active_sec', 
    'gm_dailies_distance', 'gm_dailies_moderate_sec', 
 
    # Affective Dysregulation 
    'garmin_hrv_mean_ep_0', 'gm_dailies_activity_stress_duration', 
    'gm_dailies_average_stress', 'garmin_stress_mean_ep_0', 
    'gm_dailies_high_stress_duration', 'gm_dailies_low_stress_duration', 
    'gm_dailies_max_stress', 'gm_dailies_medium_stress_duration', 
 
    # Behavioral Inactivation 
    'unlock_duration_ep_0', 'unlock_num_ep_0', 'home_ep_0', 
    'loc_visit_num_ep_0', 'loc_dist_ep_0', 
 
    # Social Withdrawal 
    'audio_convo_duration_ep_0', 'audio_convo_num_ep_0', 
    'call_in_duration_ep_0', 'call_in_num_ep_0', 
    'call_out_duration_ep_0', 'call_out_num_ep_0', 
    'sms_in_num_ep_0', 'sms_out_num_ep_0' 
] 

id_cols = ['uid', 'day'] 
keep_cols = id_cols + variables_table2 
df = df[[col for col in keep_cols if col in df.columns]] 

#df[['gm_sleep_duration']] = abs(8-df[['gm_sleep_duration']]) # I tried this as robustness check to account for too much or too little sleep, can be ignored
#df[['sleep_duration']] = abs(8-df[['sleep_duration']])

# Remove Implausible Dates 
df['day'] = pd.to_datetime(df['day'], errors='coerce') 
df = df[df['day'].dt.year > 2010]  # remove 1970 or other nvalid years 
print(len(np.unique(df[df['uid'].str.startswith('t')]["uid"])))# 10
df = df[~df['uid'].str.startswith('t')] # remove test participants

# Label Fully Zero Rows as Missing (not existing here) 
df['all_vars_zero'] = df.drop(columns=['day', 'uid']).eq(0).all(axis=1)
df.loc[df['all_vars_zero'], variables_table2] = np.nan 
df.drop(columns='all_vars_zero', inplace=True) 
df[df[variables_table2].isna().all(axis=1)]

# Remove Participants with All Missing Values (if all equals zero)
df['row_missing'] = df[variables_table2].isna().all(axis=1) 
missing_per_participant = df.groupby('uid')['row_missing'].all() 
participants_to_drop = missing_per_participant[missing_per_participant].index 
print(participants_to_drop)
df = df[~df['uid'].isin(participants_to_drop)] 
df.drop(columns='row_missing', inplace=True) 
df = df.reset_index()

# Exclude participants who have less than 45 days of sensor data
uid_counts = df.groupby("uid").count()["day"] 
uids_to_exclude = uid_counts[uid_counts < 45].index
print(len(uids_to_exclude))
df = df[~df['uid'].isin(uids_to_exclude)]
df = df.reset_index(drop=True)

# Add numeric day, account for missing days, and account for weird breaks
# Some participants have a certain start date but then pause for 2 weeks, Thus I exclude the first day and they need 3 days of data

df["day"] = pd.to_datetime(df.day, format="%Y-%m-%d")

uids = list(set(df["uid"]))
ESM_normalized = []

for j in range(len(uids)):
    ESM_part = df.loc[df["uid"] == uids[j]]
    ESM_part = ESM_part.reset_index(drop = True)

    Dates = pd.DataFrame({"day": np.unique(ESM_part.day)})
    time_range_date = pd.date_range(min(Dates.day),min(Dates.day) + timedelta(days=300), freq='D') # I just put a high number of dates here
    time_range_date = pd.DataFrame({'day':time_range_date, 'uid':uids[j]})
    ESM_part = pd.merge(ESM_part,time_range_date,how = 'right') 
    ESM_part = ESM_part.reset_index(drop = True)
    
    # Cut off first days if they are without data (three subsequent days need to have at least data
    valid_rows = ESM_part[variables_table2].notna().all(axis=1)
    rolling_valid = valid_rows.rolling(3).sum().shift(-2)
    
    # Find the first index where 3 consecutive rows have all valid data
    start_index = rolling_valid[rolling_valid == 3].index.min()
    if pd.notna(start_index):
        ESM_part = ESM_part[ESM_part.index >= start_index].reset_index(drop=True)

                                                       
    Dates = pd.DataFrame({"day": np.unique(ESM_part.day)})
    time_range_date = pd.date_range(min(Dates.day),min(Dates.day) + timedelta(days=90), freq='D') #create range 1 to 90
    time_range_date = pd.DataFrame({'day':time_range_date, 'uid':uids[j]})
    ESM_part = pd.merge(ESM_part,time_range_date,how = 'right') 
    ESM_part = ESM_part.reset_index(drop = True)

    ESM_normalized.append(ESM_part)
    
df = pd.concat(ESM_normalized, ignore_index=True)
df['day_num'] = df.groupby('uid').cumcount() + 1
df = df.sort_values(by=['uid', 'day'])

df = df[df['day_num'] <= 90] # Only use first 90 days (to avoid that some participants are overpresented)

def reverse(df, reverse_items):

    df_with_scores = df.copy()

    # Automatically reverse code specified items using their min/max in the dataframe
    for col in reverse_items:
        if col in df_with_scores.columns:
            col_min = df_with_scores[col].min()
            col_max = df_with_scores[col].max()
            df_with_scores[col] = (col_max + col_min) - df_with_scores[col]

  
    return df_with_scores

  
reverse_items = [
    # Higher Sleep Disturbance (increase too much/little sleep, increased awake, increased rem, less deep, less sleep quality)
    "gm_sleep_quality", "gm_sleep_duration_deep",

    # Lower Activity (lower steps, increased still, lower kcal, lower activity sec, lower distance, lower moderate sec)
    "gm_dailies_step", "step_count2", "garmin_steps",
    "gm_dailies_active_kcal", "gm_dailies_active_sec", 
    "gm_dailies_distance", "gm_dailies_moderate_sec",

    # Higher Affective Dysregulation (lower hrv, increased stress)
    "garmin_hrv_mean_ep_0",

    # Higher Behavioral Inactivation (higher unlock, higher home, lower locations)
    "loc_visit_num_ep_0", "loc_dist_ep_0","unlock_num_ep_0",

    # Lower Social
    'audio_convo_duration_ep_0', 'audio_convo_num_ep_0', 
    'call_in_duration_ep_0', 'call_in_num_ep_0', 
    'call_out_duration_ep_0', 'call_out_num_ep_0', 
    'sms_in_num_ep_0', 'sms_out_num_ep_0'
]

#reverse_items = []

df = reverse(df, reverse_items)

# Save Cleaned Dataset  
df.to_csv("/Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/Daily_Cleaned_Dataset.csv", index=False) 
#df.to_csv("/Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/Daily_Cleaned_Dataset_nonreverse.csv", index=False) 


print("Daily_Cleaned_Dataset.csv created.")

10
Index([], dtype='object', name='uid')
9
Daily_Cleaned_Dataset.csv created.


In [45]:
import pandas as pd


#313, 10 Test IDs
#303, 9 had less than 45 days of data
#294


# Load and prepare data
df = pd.read_csv("/Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/Daily_Cleaned_Dataset.csv")
len(np.unique(df["uid"]))

294

In [46]:
### #Step 2 
df = pd.read_csv("/Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/Daily_Cleaned_Dataset.csv") 
#df = pd.read_csv("/Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/Daily_Cleaned_Dataset_nonreverse.csv") 
 
df['day'] = pd.to_datetime(df['day']) 
df = df.sort_values(by=['uid', 'day']) 
 
# List of variables to aggregate (exclude id and date cols) 
exclude_cols = ['uid', 'day'] 
value_vars = [col for col in df.columns if col not in exclude_cols] 

### Weekly Aggregation ###
# Create a participant-relative week index 
df['week_num'] = df.groupby('uid').cumcount() // 7 + 1

# Group by participant and week, then compute the mean 
weekly_df = df.groupby(['uid', 'week_num'])[value_vars].mean().reset_index() 
weekly_df.to_csv("/Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/Weekly_Cleaned_Dataset.csv", index=False) 
#weekly_df.to_csv("/Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/Weekly_Cleaned_Dataset_nonreverse.csv", index=False) 

print("Weekly_Cleaned_Dataset.csv saved.") 
 
### Monthly Aggregation ###
 
# Create a participant-relative month index (each 30 days = 1 month) 
df['month_num'] = df.groupby('uid').cumcount() // 30 + 1

# Group by participant and month, then compute the mean 
monthly_df = df.groupby(['uid', 'month_num'])[value_vars].mean().reset_index() 
monthly_df.to_csv("/Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/Monthly_Cleaned_Dataset.csv", index=False) 
#monthly_df.to_csv("/Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/Monthly_Cleaned_Dataset_nonreverse.csv", index=False) 

print("Monthly_Cleaned_Dataset.csv saved.") 


Weekly_Cleaned_Dataset.csv saved.
Monthly_Cleaned_Dataset.csv saved.


In [47]:
### Step 3
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

scaler = "minmax"
#scaler = "standard"

datasets = {
    "Daily": "/Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/Daily_Cleaned_Dataset.csv",
    "Weekly": "/Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/Weekly_Cleaned_Dataset.csv",
    "Monthly": "/Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/Monthly_Cleaned_Dataset.csv"
}

def apply_log_transform(df):
    
    return np.log1p(df)

def apply_minmax_scaling(df):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)
    return pd.DataFrame(scaled_data, columns=df.columns)

for name, filename in datasets.items():
    df = pd.read_csv(filename)
    #df = df.replace([np.inf, -np.inf], np.nan)
    #df = df.dropna()

    # Log + Min-Max Scaling
    log_transformed = df.copy()
    log_transformed[variables_table2] = np.log1p(
        log_transformed[variables_table2].clip(lower=0)  # avoid negatives
    )
    log_minmax_scaled = log_transformed.copy()
    log_minmax_scaled[variables_table2] = apply_minmax_scaling(log_minmax_scaled[variables_table2])
    log_minmax_scaled.to_csv(f"/Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/{name}_Cleaned_Dataset_Log_{scaler}.csv", index=False)

    # Min-Max Scaling Only
    minmax_scaled_only = df.copy()
    minmax_scaled_only[variables_table2] = apply_minmax_scaling(minmax_scaled_only[variables_table2])
    minmax_scaled_only.to_csv(f"/Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/{name}_Cleaned_Dataset_{scaler}.csv", index=False)

print("Transformation complete. Files saved.")


Transformation complete. Files saved.


In [48]:
# Construct groups
constructs = {
    'Sleep_Score': [
        'gm_sleep_duration', 'gm_sleep_duration_awake', 
        'gm_sleep_duration_deep', 'gm_sleep_duration_rem', 
        'gm_sleep_quality',"sleep_duration"
    ],
    'Activity_Score': [
        'gm_dailies_step', 'step_count2', 
        'garmin_steps', 'act_still_ep_0', 'gm_dailies_active_kcal', 
        'gm_dailies_active_sec', 'gm_dailies_distance', 
        'gm_dailies_moderate_sec'
    ],
    'AffectiveDysregulation_Score': [
        'garmin_hrv_mean_ep_0', 'gm_dailies_activity_stress_duration', 
        'gm_dailies_average_stress', 'garmin_stress_mean_ep_0', 
        'gm_dailies_high_stress_duration', 'gm_dailies_low_stress_duration', 
        'gm_dailies_max_stress', 'gm_dailies_medium_stress_duration'
    ],
    'BehavioralInactivation_Score': [
        'unlock_duration_ep_0', 'unlock_num_ep_0', 'home_ep_0', 
        'loc_visit_num_ep_0', 'loc_dist_ep_0'
    ],
    'SocialWithdrawal_Score': [
        'audio_convo_duration_ep_0', 'audio_convo_num_ep_0', 
        'call_in_duration_ep_0', 'call_in_num_ep_0', 
        'call_out_duration_ep_0', 'call_out_num_ep_0', 
        'sms_in_num_ep_0', 'sms_out_num_ep_0'
    ]
}

# Dataset base names
dataset_types = ['Daily', 'Weekly', 'Monthly']
transforms = ['Log_MinMax', 'MinMax']
#transforms = ['Log_Standard', 'Standard']

def add_construct_scores(df, constructs):
    df_with_scores = df.copy()
    # Recode reverse coded items
    
    # # Higher Sleep Disturbance (increase too much/little sleep, increased awake, increased rem, less deep, less sleep quality)
    # df_with_scores["gm_sleep_quality"] = 1 - df_with_scores["gm_sleep_quality"]
    # df_with_scores["gm_sleep_duration_deep"] = 1 - df_with_scores["gm_sleep_duration_deep"]
    
    # # Lower Activity (lower steps, increased still, lower kcal, lower activity sec, lower distance, lower moderate sec)
    # df_with_scores["gm_dailies_step"] = 1 - df_with_scores["gm_dailies_step"]
    # df_with_scores["step_count2"] = 1 - df_with_scores["step_count2"]
    # df_with_scores["garmin_steps"] = 1 - df_with_scores["garmin_steps"]
    # df_with_scores["gm_dailies_active_kcal"] = 1 - df_with_scores["gm_dailies_active_kcal"]
    # df_with_scores["gm_dailies_active_sec"] = 1 - df_with_scores["gm_dailies_active_sec"]
    # df_with_scores["gm_dailies_distance"] = 1 - df_with_scores["gm_dailies_distance"]
    # df_with_scores["gm_dailies_moderate_sec"] = 1 - df_with_scores["gm_dailies_moderate_sec"]
    # # double check what activity still means
    
    # # Higher Affective Dysregulation (lower hrv, increased stress)
    # df_with_scores["garmin_hrv_mean_ep_0"] = 1 - df_with_scores["garmin_hrv_mean_ep_0"]
    
    # # Higher Behavioral Inactivation (higher unlock, higher home, lower locations)
    # df_with_scores["loc_visit_num_ep_0"] = 1 - df_with_scores["loc_visit_num_ep_0"]
    # df_with_scores["loc_dist_ep_0"] = 1 - df_with_scores["loc_dist_ep_0"]
    # df_with_scores["unlock_num_ep_0"] = 1 - df_with_scores["unlock_num_ep_0"]
    
    # # Lower Social
    # for col in [ 'audio_convo_duration_ep_0', 'audio_convo_num_ep_0', 
    #     'call_in_duration_ep_0', 'call_in_num_ep_0', 
    #     'call_out_duration_ep_0', 'call_out_num_ep_0', 
    #     'sms_in_num_ep_0', 'sms_out_num_ep_0'
    # ]:
    #     df_with_scores[col] = 1 - df_with_scores[col]


    
    for score_name, cols in constructs.items():
        existing_cols = [col for col in cols if col in df.columns]
        df_with_scores[score_name] = df[existing_cols].mean(axis=1, skipna=True)
    return df_with_scores



# Process and save updated datasets
for dataset in dataset_types:
    for transform in transforms:
        filename = f"/Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/{dataset}_Cleaned_Dataset_{transform}.csv"
        try:
            df = pd.read_csv(filename)
            df_with_scores = add_construct_scores(df, constructs)
            output_name = f"/Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/{dataset}_Cleaned_Dataset_{transform}_Complete.csv"
            df_with_scores.to_csv(output_name, index=False)
            print(f"Saved: {output_name}")
        except FileNotFoundError:
            print(f"File not found: {filename}")


Saved: /Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/Daily_Cleaned_Dataset_Log_MinMax_Complete.csv
Saved: /Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/Daily_Cleaned_Dataset_MinMax_Complete.csv
Saved: /Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/Weekly_Cleaned_Dataset_Log_MinMax_Complete.csv
Saved: /Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/Weekly_Cleaned_Dataset_MinMax_Complete.csv
Saved: /Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/Darmouth Drive/4_Reliability Project_Data/Monthly_Cleaned_Dataset_Log_MinMax_Complete.csv
Saved: /Users/f007qrc/Library/CloudStorage/GoogleDrive-anna.m.langener@dartmouth.edu/My Drive/